In [68]:
import numpy as np
from scipy.spatial.transform import Rotation
import scipy.special
#import scipy.optimize
from scipy.stats import chi2
import lmfit as lm
import pandas as pd
#import pickle as pkl
#import re
#from copy import deepcopy
import matplotlib.pyplot as plt
%matplotlib inline

#import plotly.express as px

In [69]:
def euler_model(phis, thetas, **params):
    alpha0,beta0,gamma0 = params['alpha0'], params['beta0'], params['gamma0']
    xs = np.sin(thetas)*np.cos(phis)
    ys = np.sin(thetas)*np.sin(phis)
    zs = np.cos(thetas)
    pos = np.array([xs,ys,zs]).T
    #rot = Rotation.from_euler('XYZ', np.array([alpha0,beta0,gamma0]).flatten())
    rot = Rotation.from_euler('ZYX', -np.array([alpha0,beta0,gamma0]).flatten())
    pos_rot = rot.apply(pos)
    thetas_rot = np.arccos(pos_rot[:,2]) # cos(theta) = z / r, r=1
    phis_rot = np.arctan2(pos_rot[:,1], pos_rot[:,0]) # tan(phi) = y / x
    return np.concatenate([phis_rot, thetas_rot])

- Ensemble of fits:
    1. Randomly generate $\alpha, \beta,\gamma$
    2. Run the fit (starting from all set to 0)
- From ensemble look at:
    - Chi2
    - Parameter pulls
    - Parameter residuals
    - Correlations

In [70]:
def gen_data(phis, thetas, sigma, params):
    # create meshgrid for all input independent variables
    Ph, Th = np.meshgrid(phis, thetas)
    # flatten all to 1D arrays
    Th = Th.flatten()
    Ph = Ph.flatten()
    
    # create pandas.DataFrame to store information from each data point
    df = pd.DataFrame({"Theta": Th, "Phi": Ph})
    # add columns for angles in degrees for convenience
    df["Theta_deg"] = np.degrees(df.Theta)
    df["Phi_deg"] = np.degrees(df.Phi)
    # use forward function to generate voltage data and store in new column
    #df["V_obs"] = V_forward(df.B.values, df.Temp.values, df.Theta.values, df.Phi.values, **params)
    #phiprime, thetaprime = euler_model(Th, Ph, **params).reshape((2, -1))
    phiprime, thetaprime = euler_model(phis=Ph, thetas=Th, **params).reshape((2, -1))
    df['phi1_exact'] = phiprime
    df['theta1_exact'] = thetaprime
    # inject noise
    df["phi1_obs"] = df["phi1_exact"] + np.random.normal(loc=0.0, scale=sigma, size=len(df))
    df["theta1_obs"] = df["theta1_exact"] + np.random.normal(loc=0.0, scale=sigma, size=len(df))
    # set sigma as a column
    df["sigma_V"] = sigma
    
    return df

## Test Generation

In [6]:
alpha0 = np.random.uniform(low=-np.pi, high=np.pi, size=1)[0]
beta0 = np.random.uniform(low=0, high=np.pi, size=1)[0]
gamma0 = np.random.uniform(low=-np.pi, high=np.pi, size=1)[0]

In [71]:
alpha0, beta0, gamma0 = 0, np.pi/2, -np.pi/2

In [72]:
params_true = {'alpha0':alpha0, 'beta0':beta0, 'gamma0':gamma0}

In [73]:
params_true

{'alpha0': 0, 'beta0': 1.5707963267948966, 'gamma0': -1.5707963267948966}

In [74]:
df_gen = gen_data(np.linspace(0, 2*np.pi, 200), np.linspace(0, np.pi, 100), 0.01, params_true)

In [75]:
df_gen

,Theta,Phi,Theta_deg,Phi_deg,phi1_exact,theta1_exact,phi1_obs,theta1_obs,sigma_V
0,0.000000,0.000000,0.0,0.000000,-1.570796,1.570796,-1.578516,1.562778,0.01
1,0.000000,0.031574,0.0,1.809045,-1.570796,1.570796,-1.586900,1.589680,0.01
2,0.000000,0.063148,0.0,3.618090,-1.570796,1.570796,-1.574841,1.570347,0.01
3,0.000000,0.094721,0.0,5.427136,-1.570796,1.570796,-1.559290,1.578382,0.01
4,0.000000,0.126295,0.0,7.236181,-1.570796,1.570796,-1.561856,1.549848,0.01
...,...,...,...,...,...,...,...,...,...
19995,3.141593,6.156890,180.0,352.763819,1.570796,1.570796,1.558396,1.552900,0.01
19996,3.141593,6.188464,180.0,354.572864,1.570796,1.570796,1.586788,1.557458,0.01
19997,3.141593,6.220038,180.0,356.381910,1.570796,1.570796,1.580907,1.566940,0.01
19998,3.141593,6.251612,180.0,358.190955,1.570796,1.570796,1.567899,1.552473,0.01


In [76]:
df_test = df_gen[np.isin(df_gen['Theta'],df_gen['Theta'].unique()[::10])
                 & np.isin(df_gen['Phi'],df_gen['Phi'].unique()[::10])]

In [77]:
df_test

,Theta,Phi,Theta_deg,Phi_deg,phi1_exact,theta1_exact,phi1_obs,theta1_obs,sigma_V
0,0.000000,0.000000,0.000000,0.000000,-1.570796,1.570796,-1.578516,1.562778,0.01
10,0.000000,0.315738,0.000000,18.090452,-1.570796,1.570796,-1.568385,1.557102,0.01
20,0.000000,0.631476,0.000000,36.180905,-1.570796,1.570796,-1.547928,1.599842,0.01
30,0.000000,0.947214,0.000000,54.271357,-1.570796,1.570796,-1.571755,1.580903,0.01
40,0.000000,1.262952,0.000000,72.361809,-1.570796,1.570796,-1.576435,1.566619,0.01
...,...,...,...,...,...,...,...,...,...
18150,2.855993,4.736069,163.636364,271.356784,1.285273,1.564125,1.291846,1.555125,0.01
18160,2.855993,5.051807,163.636364,289.447236,1.300688,1.476859,1.298168,1.470270,0.01
18170,2.855993,5.367545,163.636364,307.537688,1.342040,1.398287,1.338940,1.396672,0.01
18180,2.855993,5.683283,163.636364,325.628141,1.406520,1.336108,1.401325,1.339531,0.01


In [79]:
samples = np.concatenate([df_test['phi1_obs'].values,df_test['theta1_obs'].values])
#samples = np.concatenate([df_test['phi1_exact'].values,df_test['theta1_exact'].values])
samples.shape

(400,)

In [83]:
# construct model
model = lm.Model(euler_model, independent_vars=['phis', 'thetas'])
params = lm.Parameters()
params.add('alpha0', vary=True, value=params_true['alpha0'], min=-np.pi, max=np.pi)
params.add('beta0', vary=True, value=params_true['beta0'], min=0, max=np.pi)
params.add('gamma0', vary=True, value=params_true['gamma0'], min=-np.pi, max=np.pi)
#params.add('alpha0', vary=True, value=0, min=-np.pi, max=np.pi)
#params.add('beta0', vary=True, value=0, min=0, max=np.pi)
#params.add('gamma0', vary=True, value=0, min=-np.pi, max=np.pi)
result = model.fit(samples, 
                   phis=df_test['Phi'].values, thetas=df_test['Theta'].values,
                   params=params, weights= 1/df_test['sigma_V'].values[0], method='least_squares',
                  )#fit_kws={'max_nfev':1})#_squares')

In [84]:
result

In [ ]:
cos(alpha + gamma), sin(alpha + gamma)

In [85]:
-2.499+.927

-1.572

In [82]:
params_true

{'alpha0': 0, 'beta0': 1.5707963267948966, 'gamma0': -1.5707963267948966}

In [20]:
33761.7424 * .01

337.617424

In [21]:
phis, theta

NameError: name 'phis' is not defined

In [22]:
p

NameError: name 'p' is not defined

In [23]:
np.degrees(0.01)

0.5729577951308232

# Quaternions

In [86]:
#rot = Rotation.from_euler('ZYX', -np.array([-np.pi/2,np.pi/2,0]))
rot = Rotation.from_euler('XYZ', np.array([0,np.pi/2,-np.pi/2]))

In [87]:
rot.as_quat()

array([-0.5,  0.5, -0.5,  0.5])

In [88]:
rot.as_matrix()

array([[ 0.00000000e+00,  2.22044605e-16,  1.00000000e+00],
       [-1.00000000e+00,  2.22044605e-16,  0.00000000e+00],
       [-2.22044605e-16, -1.00000000e+00,  2.22044605e-16]])

In [89]:
def quat_model(phis, thetas, **params):
    x0,y0,z0,w0 = params['x0'], params['y0'], params['z0'], params['w0'] # let scipy normalize input
    #x0,y0,z0 = params['x0'], params['y0'], params['z0']
    #w0 = (1-(x0**2 + y0**2 + z0**2))**(1/2) # forces w0 positive
    xs = np.sin(thetas)*np.cos(phis)
    ys = np.sin(thetas)*np.sin(phis)
    zs = np.cos(thetas)
    pos = np.array([xs,ys,zs]).T
    #rot = Rotation.from_euler('XYZ', np.array([alpha0,beta0,gamma0]).flatten())
    #rot = Rotation.from_euler('ZYX', -np.array([alpha0,beta0,gamma0]).flatten())
    rot = Rotation.from_quat([x0,y0,z0,w0])
    pos_rot = rot.apply(pos)
    thetas_rot = np.arccos(pos_rot[:,2]) # cos(theta) = z / r, r=1
    phis_rot = np.arctan2(pos_rot[:,1], pos_rot[:,0]) # tan(phi) = y / x
    return np.concatenate([phis_rot, thetas_rot])

In [90]:
def gen_data_quat(phis, thetas, sigma, params):
    # create meshgrid for all input independent variables
    Ph, Th = np.meshgrid(phis, thetas)
    # flatten all to 1D arrays
    Th = Th.flatten()
    Ph = Ph.flatten()
    
    # create pandas.DataFrame to store information from each data point
    df = pd.DataFrame({"Theta": Th, "Phi": Ph})
    # add columns for angles in degrees for convenience
    df["Theta_deg"] = np.degrees(df.Theta)
    df["Phi_deg"] = np.degrees(df.Phi)
    # use forward function to generate voltage data and store in new column
    #df["V_obs"] = V_forward(df.B.values, df.Temp.values, df.Theta.values, df.Phi.values, **params)
    #phiprime, thetaprime = euler_model(Th, Ph, **params).reshape((2, -1))
    phiprime, thetaprime = quat_model(phis=Ph, thetas=Th, **params).reshape((2, -1))
    df['phi1_exact'] = phiprime
    df['theta1_exact'] = thetaprime
    # inject noise
    df["phi1_obs"] = df["phi1_exact"] + np.random.normal(loc=0.0, scale=sigma, size=len(df))
    df["theta1_obs"] = df["theta1_exact"] + np.random.normal(loc=0.0, scale=sigma, size=len(df))
    # set sigma as a column
    df["sigma_V"] = sigma
    
    return df

## Test Generation

In [91]:
x0, y0, z0, w0 = [-0.5,  0.5, -0.5,  0.5]

In [92]:
params_true = {'x0':x0, 'y0':y0, 'z0':z0, 'w0':w0}

In [93]:
params_true

{'x0': -0.5, 'y0': 0.5, 'z0': -0.5, 'w0': 0.5}

In [94]:
df_gen = gen_data_quat(np.linspace(0, 2*np.pi, 200), np.linspace(0, np.pi, 100), 0.01, params_true)

In [95]:
df_gen

,Theta,Phi,Theta_deg,Phi_deg,phi1_exact,theta1_exact,phi1_obs,theta1_obs,sigma_V
0,0.000000,0.000000,0.0,0.000000,0.000000,1.570796,-0.018248,1.577416,0.01
1,0.000000,0.031574,0.0,1.809045,0.000000,1.570796,0.009166,1.568633,0.01
2,0.000000,0.063148,0.0,3.618090,0.000000,1.570796,-0.006517,1.569779,0.01
3,0.000000,0.094721,0.0,5.427136,0.000000,1.570796,0.024550,1.559845,0.01
4,0.000000,0.126295,0.0,7.236181,0.000000,1.570796,0.006481,1.584892,0.01
...,...,...,...,...,...,...,...,...,...
19995,3.141593,6.156890,180.0,352.763819,-3.141593,1.570796,-3.132158,1.576826,0.01
19996,3.141593,6.188464,180.0,354.572864,-3.141593,1.570796,-3.162591,1.583566,0.01
19997,3.141593,6.220038,180.0,356.381910,-3.141593,1.570796,-3.158117,1.570255,0.01
19998,3.141593,6.251612,180.0,358.190955,-3.141593,1.570796,-3.143114,1.573984,0.01


In [96]:
df_test = df_gen[np.isin(df_gen['Theta'],df_gen['Theta'].unique()[::10])
                 & np.isin(df_gen['Phi'],df_gen['Phi'].unique()[::10])]

In [97]:
df_test

,Theta,Phi,Theta_deg,Phi_deg,phi1_exact,theta1_exact,phi1_obs,theta1_obs,sigma_V
0,0.000000,0.000000,0.000000,0.000000,0.000000,1.570796,-0.018248,1.577416,0.01
10,0.000000,0.315738,0.000000,18.090452,0.000000,1.570796,0.007491,1.563399,0.01
20,0.000000,0.631476,0.000000,36.180905,0.000000,1.570796,0.003111,1.564711,0.01
30,0.000000,0.947214,0.000000,54.271357,0.000000,1.570796,0.005834,1.570352,0.01
40,0.000000,1.262952,0.000000,72.361809,0.000000,1.570796,-0.006374,1.572504,0.01
...,...,...,...,...,...,...,...,...,...
18150,2.855993,4.736069,163.636364,271.356784,-3.134640,1.285279,-3.132873,1.274939,0.01
18160,2.855993,5.051807,163.636364,289.447236,-3.044143,1.301909,-3.064533,1.290534,0.01
18170,2.855993,5.367545,163.636364,307.537688,-2.964564,1.345494,-2.945220,1.335822,0.01
18180,2.855993,5.683283,163.636364,325.628141,-2.903821,1.411063,-2.887188,1.413076,0.01


In [98]:
samples = np.concatenate([df_test['phi1_obs'].values,df_test['theta1_obs'].values])
#samples = np.concatenate([df_test['phi1_exact'].values,df_test['theta1_exact'].values])
samples.shape

(400,)

In [113]:
# construct model
model = lm.Model(quat_model, independent_vars=['phis', 'thetas'])
params = lm.Parameters()
#params.add('x0', vary=True, value=-.4, min=-1, max=1)
#params.add('y0', vary=False, value=0, min=-1, max=1)
#params.add('z0', vary=False, value=0, min=-1, max=1)
#params.add('w0', vary=False, value=0, min=-1, max=1)
#params.add('x0', vary=True, value=-.5, min=-1, max=1)
#params.add('y0', vary=True, value=.5, min=-1, max=1)
#params.add('z0', vary=True, value=-.5, min=-1, max=1)
#params.add('w0', vary=True, value=.5, min=-1, max=1)
params.add('x0', vary=True, value=-.5+np.random.normal(loc=0,scale=0.01,size=1)[0], min=-.5-.1, max=-.5+.1)
params.add('y0', vary=True, value=.5+np.random.normal(loc=0,scale=0.01,size=1)[0], min=.5-.1, max=.5+.1)
params.add('z0', vary=True, value=-.5+np.random.normal(loc=0,scale=0.01,size=1)[0], min=-.5-.1, max=-.5+.1)
params.add('w0', vary=True, value=.5+np.random.normal(loc=0,scale=0.01,size=1)[0], min=.5-.1, max=.5+.1)
result = model.fit(samples, 
                   phis=df_test['Phi'].values, thetas=df_test['Theta'].values,
                   params=params, weights= 1/df_test['sigma_V'].values[0], method='least_squares',
                  )#fit_kws={'max_nfev':1})#_squares')

In [114]:
#x0, y0, z0 = [-1, 1]
#x0=1, y0=1, z0=1
#w0 = (1 - x0**2 - y0**2 - z0**2)

In [115]:
result

In [104]:
params_true

{'x0': -0.5, 'y0': 0.5, 'z0': -0.5, 'w0': 0.5}

In [20]:
33761.7424 * .01

337.617424

In [21]:
phis, theta

NameError: name 'phis' is not defined

In [22]:
p

NameError: name 'p' is not defined

In [23]:
np.degrees(0.01)

0.5729577951308232